In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import resize
import numpy as np
import pickle
import joblib

## Functions for predict and empty or not and get boxes of solts

In [2]:
EMPTY = True
NOT_EMPTY = False

import joblib

MODEL = joblib.load(r"C:\Users\youssef\Documents\GitHub\Parking-availiable-slots\svm_parking_model_rgb_15x15_new.joblib")
print("Model loaded successfully.")

def Empty_or_not(spot_bgr):

    flat_data = []

    img_resized = resize(spot_bgr, (15, 15, 3))
    flat_data.append(img_resized.flatten())
    flat_data = np.array(flat_data)

    y_output = MODEL.predict(flat_data)

    if y_output == 0:
        return EMPTY
    else:
        return NOT_EMPTY


def get_parking_spots_bboxes(connected_components):
    (totalLabels, label_ids, values, centroid) = connected_components
    #print(values)

    slots = []
    coef = 1
    for i in range(1, totalLabels):

        # Now extract the coordinate points
        x1 = int(values[i, cv2.CC_STAT_LEFT] * coef)
        y1 = int(values[i, cv2.CC_STAT_TOP] * coef)
        w = int(values[i, cv2.CC_STAT_WIDTH] * coef)
        h = int(values[i, cv2.CC_STAT_HEIGHT] * coef)

        slots.append([x1, y1, w, h])

    return slots

Model loaded successfully.


In [3]:
def calc_diff(im1, im2):
    return np.abs(np.mean(im1) - np.mean(im2))

mask_path = r"C:\Users\youssef\Documents\GitHub\Parking-availiable-slots\mask_1920_1080.png"
video_path = r"C:\Users\youssef\Documents\GitHub\Parking-availiable-slots\parking_1920_1080.mp4"


mask = cv2.imread(mask_path, 0)

cap = cv2.VideoCapture(video_path)

connected_components = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)

spots = get_parking_spots_bboxes(connected_components)
print(spots)

[[533, 98, 65, 35], [991, 109, 64, 37], [611, 115, 63, 35], [1068, 127, 64, 35], [146, 137, 68, 27], [306, 136, 61, 26], [387, 136, 61, 26], [1674, 148, 71, 31], [1448, 150, 68, 33], [1518, 150, 68, 31], [1221, 152, 68, 33], [1291, 152, 69, 32], [534, 154, 64, 33], [387, 165, 61, 30], [837, 165, 69, 26], [992, 165, 64, 35], [307, 166, 62, 31], [763, 166, 72, 26], [147, 169, 68, 27], [613, 170, 61, 36], [1674, 181, 70, 29], [1069, 182, 64, 34], [1518, 183, 69, 30], [1448, 184, 68, 30], [1221, 187, 69, 29], [1292, 186, 68, 29], [1776, 186, 62, 26], [763, 193, 72, 28], [837, 193, 69, 27], [388, 199, 60, 30], [307, 200, 62, 29], [147, 203, 68, 27], [535, 208, 65, 36], [1675, 212, 69, 29], [1448, 215, 69, 29], [1519, 214, 68, 29], [1776, 214, 62, 29], [1292, 217, 68, 29], [992, 219, 71, 29], [1065, 219, 69, 29], [1221, 218, 69, 29], [837, 221, 69, 29], [764, 222, 71, 30], [613, 225, 62, 24], [309, 233, 68, 30], [381, 232, 68, 31], [148, 237, 68, 28], [1675, 243, 69, 30], [1448, 245, 69, 30]

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Initialize spot status and differences lists
spots_status = [None for _ in spots]
diffs = [None for _ in spots]

previous_frame = None
frame_nmr = 0
ret = True
step = 30

# Start video capture loop
while ret:
    ret, frame = cap.read()

    if frame_nmr % step == 0 and previous_frame is not None:
        for spot_indx, spot in enumerate(spots):
            x1, y1, w, h = spot
            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]
            diffs[spot_indx] = calc_diff(spot_crop, previous_frame[y1:y1 + h, x1:x1 + w, :])

        # Display histogram of differences
        sorted_diffs = [diffs[j] / np.amax(diffs) for j in np.argsort(diffs)][::-1]
        print(sorted_diffs)
        plt.hist(sorted_diffs, bins=20)
        plt.show()

    if frame_nmr % step == 0:
        if previous_frame is None:
            arr_ = range(len(spots))
        else:
            arr_ = [j for j in np.argsort(diffs) if diffs[j] / np.amax(diffs) > 0.4]

        for spot_indx in arr_:
            x1, y1, w, h = spots[spot_indx]
            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]
            spot_status = Empty_or_not(spot_crop)
            spots_status[spot_indx] = spot_status

    if frame_nmr % step == 0:
        previous_frame = frame.copy()

    # Draw rectangles for each spot based on its status
    for spot_indx, spot in enumerate(spots):
        x1, y1, w, h = spot
        color = (0, 255, 0) if spots_status[spot_indx] else (0, 0, 255)
        frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), color, 2)

    # Display available spots
    cv2.rectangle(frame, (80, 20), (550, 80), (0, 0, 0), -1)
    cv2.putText(frame, f'Available spots: {sum(spots_status)} / {len(spots_status)}', 
                (100, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Display the frame in a resized window
    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('frame', 640, 480)  # Adjusted window size
    cv2.imshow('frame', frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    frame_nmr += 1

cap.release()
cv2.destroyAllWindows()